In [3]:
# Dependencies and setup
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import matplotlib.pyplot as plt

from api_keys import census_key

In [15]:
# Import unemployment rate csv file, create DataFrame, and set index to Area Type
unemployment_data = Path('resources/Local_Area_Unemployment_Statistics__LAUS___Annual_Average.csv')
unemployment_df = pd.read_csv(unemployment_data)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment','Unemployment Rate']]
area_type_df

# Group by Year and calculate averages
year_grouped = area_type_df.groupby(['Year', 'Area Name']).mean()
year_grouped

# Reduce to only 2017-2022
recent = year_grouped.loc[[2017, 2018, 2019, 2020, 2021, 2022]]
recent

,Area Name,Year,Period,Labor Force,Employment,Unemployment,Unemployment Rate,Seasonally Adjusted (Y N),Status
Area Type,,,,,,,,,
State,California,1990,Annual,15141700,14267900,873800,5.8,N,Final
County,Alameda County,1990,Annual,677200,649800,27400,4.0,N,Final
County,Alpine County,1990,Annual,380,340,40,9.6,N,Final
County,Amador County,1990,Annual,11450,10860,590,5.2,N,Final
County,Butte County,1990,Annual,78800,72200,6600,8.4,N,Final
...,...,...,...,...,...,...,...,...,...
Sub-County Place,Yountville town,2022,Annual,1500,1400,100,4.9,N,Final
Sub-County Place,Yreka city,2022,Annual,2940,2850,90,3.1,N,Final
Sub-County Place,Yuba City city,2022,Annual,32000,29900,2100,6.6,N,Final


In [5]:
# Import age csv file and create DataFrame
age_data = Path('resources/experiencing_homelessness_age_demographics.csv')
age_df = pd.read_csv(age_data)
age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [6]:
# Create clean DataFrame by dropping rows with '*' in count column
clean_age_df = age_df.loc[age_df['EXPERIENCING_HOMELESSNESS_CNT']!='*']
clean_age_df

,CALENDAR_YEAR,COC_ID,COC_NAME,AGE_GROUP_PUBLIC,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,18-24,15862
1,2017,All,California,25-34,28339
2,2017,All,California,35-44,25556
3,2017,All,California,45-54,27441
4,2017,All,California,55-64,23220
...,...,...,...,...,...
2136,2022,CA-614,San Luis Obispo County CoC,45-54,389
2137,2022,CA-614,San Luis Obispo County CoC,55-64,360
2138,2022,CA-614,San Luis Obispo County CoC,65+,174
2139,2022,CA-614,San Luis Obispo County CoC,Under 18,742


In [47]:
total_homeless_count = clean_age_df.groupby(['CALENDAR_YEAR', 'COC_NAME'])['EXPERIENCING_HOMELESSNESS_CNT'].sum().reset_index()
total_homeless_count

,CALENDAR_YEAR,COC_NAME,EXPERIENCING_HOMELESSNESS_CNT
0,2017,Alameda County CoC,6713
1,2017,"Alpine, Inyo, Mono Counties CoC",151
2,2017,"Amador, Calaveras, Mariposa, Tuolumne Counties...",368
3,2017,Butte County CoC,1400
4,2017,California,184234
...,...,...,...
263,2022,Stanislaus County CoC,6482
264,2022,Tehama County CoC,670
265,2022,Ventura County CoC,2919
266,2022,Yolo County CoC,1419


In [10]:
# Import ethnicity csv file and create DataFrame
ethnicity_data = Path('resources/experiencing_homelessness_ethnicity_demographics.csv')
ethnicity_df = pd.read_csv(ethnicity_data)
ethnicity_df

,CALENDAR_YEAR,COC_ID,COC_NAME,ETHNICITY,EXPERIENCING_HOMELESSNESS
0,2017,All,California,Hispanic/Latinx,59663
1,2017,All,California,Not Hispanic/Latinx,118412
2,2017,All,California,Unknown,3796
3,2017,CA-500,Santa Clara County CoC,Hispanic/Latinx,5010
4,2017,CA-500,Santa Clara County CoC,Not Hispanic/Latinx,5005
...,...,...,...,...,...
782,2022,CA-613,Imperial County CoC,Not Hispanic/Latinx,248
783,2022,CA-613,Imperial County CoC,Unknown,74
784,2022,CA-614,San Luis Obispo County CoC,Hispanic/Latinx,994
785,2022,CA-614,San Luis Obispo County CoC,Not Hispanic/Latinx,1722


In [12]:
# Import gender csv file and create DataFrame
gender_data = Path('resources/experiencing_homelessness_gender_demographics.csv')
gender_df = pd.read_csv(gender_data)
gender_df 

,CALENDAR_YEAR,COC_ID,COC_NAME,GENDER,EXPERIENCING_HOMELESSNESS_CNT
0,2017,All,California,Female,78453
1,2017,All,California,Male,101150
2,2017,All,California,Non-Singular Gender,137
3,2017,All,California,Questioning Gender,*
4,2017,All,California,Transgender,667
...,...,...,...,...,...
1229,2022,CA-614,San Luis Obispo County CoC,Female,1279
1230,2022,CA-614,San Luis Obispo County CoC,Male,1556
1231,2022,CA-614,San Luis Obispo County CoC,Non-Singular Gender,*
1232,2022,CA-614,San Luis Obispo County CoC,Transgender,*


In [14]:
# Import race csv file and create DataFrame
race_data = Path('resources/experiencing_homelessness_race_demographics.csv')
race_df = pd.read_csv(race_data)
race_df 

,CALENDAR_YEAR,COC_ID,COC_NAME,RACE,EXPERIENCING_HOMELESSNESS
0,2017,All,California,"American Indian, Alaska Native, or Indigenous",5588
1,2017,All,California,Asian or Asian American,2951
2,2017,All,California,"Black, African American, or African",57413
3,2017,All,California,Multiple Races,8919
4,2017,All,California,Native Hawaiian or Pacific Islander,2547
...,...,...,...,...,...
1840,2022,CA-614,San Luis Obispo County CoC,"Black, African American, or African",144
1841,2022,CA-614,San Luis Obispo County CoC,Multiple Races,170
1842,2022,CA-614,San Luis Obispo County CoC,Native Hawaiian or Pacific Islander,27
1843,2022,CA-614,San Luis Obispo County CoC,Unknown,166
